### Intro to starclass and RLC usage 
#### Updated 01/03/2022
## Setup
First off, I keep both <code>starclass.py</code> & <code>RLC.py</code> in a <code>"modules"</code> subfolder, all <code>"catalog_with_astrobase_periods_addon_q05.csv"</code>'s in a separate <code>"astrobase_out"</code> subfolder, and all raw light curve data within a ___zipped___ archive <code>"TFA-20210902T000721Z-001.zip"</code> (to conserve memory and storage), downloadable from the Q05 TFA LC folders in the Google Drive under a <code>"TFA_LCS"</code> subfolder. Unsurprisingly, directories will need to be changed in the .py files according to the specific locations you want to pull from. Additionally, internal functions (e.g. loading star .rlc file data) is based around string formatting to correctly source data, so be sure the formatting is compatible with local files. Please note that creation of the stardict was initially built to load __three__ quarters but I have since changed it to deal with a __single__ quarter, specifically quarter 5. Finally, if exporting data, create an empty folder (i.e. <code>"features/"</code> for the files to be stored in. Below is a diagram of the file locations:<br>
__Root:__ <code>StaRLC_usage.ipynb</code><br>
- <code>"modules/"</code>
 - <code>"starclass.py"</code>
 - <code>"RLC.py"</code>
- <code>"astrobase_out/"</code>
 - <code>"catalog_with_astrobase_periods_addon_q05.csv"</code>
- <code>"TFA_LCS/"</code>
 - <code>"TFA-20210902T000721Z-001.zip"</code><br>
- <code>"features/"</code>

With the necessary paths  changed in the .py files, we can begin by importing the two modules, pandas, and numpy:

In [1]:
from modules.starclass import MakeStardict
from modules.RLC import RLC

import pandas as pd
import numpy as np

ImportError: No module named 'modules'

## Building a stardict
### Crash Course Under-the-Hood
__To build a stardict, call MakeStardict with either a _list of Gaia IDs to create Star and RLC objects for_ or _None to create objects for all Gaia IDs available (not recommended for testing)_.__ <br>
Running MakeStardict first loads in the star catalog containing star data (Gaia ID, RA, Dec, magnitudes, and colors) from the <code>"catalog_with_astrobase_periods_addon_q05.csv"</code> file. After the general star data is pulled, it calculates three color values and sets a generator function to find neighbor IDs when called. Creating Star objects then initializes an RLC object from the provided Gaia ID and attempts to find a .rlc file in the zip archive of detrended data. __If no matching .rlc file is located, RLC time, magnitude, and error data will be defined as None, and no further attributes will be defined for the object__, and the rest of this notebook will cover cases of existing .rlc files. For stars with an existing RLC file, RLC data will be loaded before automatically running periodograms, searching for nonperiodic features, and checking for periodogram features\*. For stars with RLC data, the <code>analyze_period</code> method can be called with a period to generate phase-folded feature and fit data.
_*The GLS and Stellingwerf algorithms are compatible with each other in this function, but the BLS results had to be separated. Please see RLC.py comments for more details on this._
### StaRLC Data Structures
The core of these two modules are the Star and RLC objects. A fully defined Star object and RLC object has atrributes: <br>
- <code>Star.ID</code> _(dtype: str)_ Gaia ID,<br>
- <code>Star.ra</code> _(dtype: float)_ Right-ascension (decimal),<br>
- <code>Star.dec</code> _(dtype: float)_ Declination (decimal),<br>
- <code>Star.Gaia_mag</code> _(dtype: float)_ Average Gaia magnitude,<br>
- <code>Star.BP_mag</code> _(dtype: float)_ Average Vega BP magnitude,<br>
- <code>Star.color_BP_RP</code> _(dtype: float)_ Color: BP-RP,<br>
- <code>Star.color_BP_G</code> _(dtype: float)_ Color: BP-G,<br>
- <code>Star.color_G_RP</code> _(dtype: float)_ Color: G-RP,<br>
- <code>Star.inrad</code> _(dtype: generator object)_ Call <code>next(Star.inrad)</code> to list all neighbor Gaia IDs within a 0.0125 degree radius of this star ascending by distance ___(internal method___<code>Star.\_\_inrad__()</code> ___yields a sorted pandas index including includes this Star, index 1 is the nearest neighbor)___'
- <code>Star.RLC</code> _(dtype: RLC object)_ RLC object more below:<br>
    - <code>RLC.time</code> _(dtype: np.ndarray or None)_ Array of time data,<br>
    - <code>RLC.mag</code> _(dtype: np.ndarray or None)_ Array of magnitude data,<br>
    - <code>RLC.err</code> _(dtype: np.ndarray or None)_ Array of magnitude error data,<br>
___If no RLC file is found, the following RLC attributes will not be defined.___<br>
    - <code>RLC.nonperiodic_feats</code> _(dtype: dict)_ Dictionary of the astrobase nonperiodic features function omitting arrays,<br>
    - <code>RLC.GLS</code> _(dtype: dict)_ Results of Lomb-Scargle periodogram,<br>
    - <code>RLC.BLS</code> _(dtype: dict)_ Results of box-least square periodogram,<br>
    - <code>RLC.Stwf</code> _(dtype: dict)_ Results of Stellingwerf periodogram,<br>
    - <code>RLC.BLS_feats</code> _(dtype: dict)_ Periodogram features for BLS,<br>
    - <code>RLC.GLS_Stwf_feats</code> _(dtype: dict)_ Periodogram features of GLS and Stellingwerf algorithms (I couldn't find a way to resolve the value error from looking through documentation or otherwise)<br>
    - <code>RLC.analyze_period(period)</code> _(Method - returns dict)_ Returns flat dictionary detailing phased and fitted LC features with respect to the provided period<br>
- <code>Star.export_features(pos_data=True, mags=True, colors=True, neighbors=False, nonper_data=True)</code> _(Method - returns dict)_ Returns dictionary of dictionaries compiling the calculated features and other stellar parameters

Calling and providing MakeStardict with Star and RLC objects for WOCS 63060, a rotator/pulsator, and a detached EB:

In [2]:
%%time
# stardict=MakeStardict(['2051105479535706752', '2051287139470009088', '2051288372119626368'])
stardict=MakeStardict(['2051105479535706752'])

NameError: name 'MakeStardict' is not defined

## Usage Examples
With our newly defined stardict, let's key stardict for the infamous WOCS 63060, print some general information about it, and then display all defined attributes:

In [3]:
print(stardict['2051105479535706752'])
vars(stardict['2051105479535706752'])

NameError: name 'stardict' is not defined

With the availability of RLC data for WOCS 63060, all raw RLC data, nonperiodic features, results of periodogram algorithms, and combined periodogram features are defined and contained within the RLC attribute of the Star object. Let's take a quick gander at a complete RLC object's attributes:

In [4]:
print(stardict['2051105479535706752'].RLC)
vars(stardict['2051105479535706752'].RLC)

NameError: name 'stardict' is not defined

Let's analyze each of the best BLS, GLS, and Stellingwerf periods and print the analysis results:

In [5]:
wocs_63060=stardict['2051105479535706752']
for best in [wocs_63060.RLC.GLS, wocs_63060.RLC.Stwf, wocs_63060.RLC.BLS]:
    print(wocs_63060.RLC.analyze_period(best['bestperiod']))

NameError: name 'stardict' is not defined

__Note:__ the dictionary returned by <code>RLC.analyze_period</code> ___only___ contains the analyzed period, and the results of the two periodic feature analyses from astrobase. To view nonperiodic features we can request the attribute from the RLC object:

In [6]:
wocs_63060.RLC.nonperiodic_feats

NameError: name 'wocs_63060' is not defined

Similarly, we can directly access the results of any of the periodogram algorithms run:

In [7]:
wocs_63060.RLC.BLS

{'bestperiod': 0.7232568175965727,
 'bestlspval': 0.0022862207155785073,
 'nbestpeaks': 3,
 'nbestlspvals': [0.0022862207155785073,
  0.0017773305459861596,
  0.0016532705431525291],
 'nbestperiods': [0.7232568175965727, 1.446434238504079, 2.890431587000283],
 'lspvals': array([0.00041692, 0.00040709, 0.00036966, ..., 0.00012153, 0.00012424,
        0.0001589 ]),
 'frequencies': array([ 0.05      ,  0.05002668,  0.05005337, ..., 23.80944475,
        23.80947143, 23.80949812]),
 'periods': array([20.        , 19.98933245, 19.97867627, ...,  0.04200014,
         0.04200009,  0.04200005]),
 'blsresult': [{'power': array([0.00041692, 0.00040709, 0.00036966, ..., 0.00010594, 0.00010037,
          0.00010365]),
   'bestperiod': 0.7232568175965727,
   'bestpower': 0.0022862207155785073,
   'transdepth': 0.004650078717658983,
   'transduration': 0.4090150250417362,
   'transingressbin': 52,
   'transegressbin': 133},
  {'power': array([0.00011619, 0.00010679, 0.00011058, ..., 0.00012153, 0.000

Taking a peak at the (self-including) list of sources within 0.0125 degrees of WOCS 63060:

In [8]:
next(wocs_63060.inrad) # To view individual neighbors, add a subscript to this command and use as key of stardict

Index(['2051105479535706752', '2051105479536285184', '2051105479530880768',
       '2051105479535709312', '2051105479535709568', '2051105479535700864',
       '2051105479535704960', '2051105479530856192', '2051105475245084928',
       '2051105479535704192', '2051105479535697024', '2051105475236399744',
       '2051105479535695232', '2051105479530849920', '2051105479535993344',
       '2051105479535697152', '2051105475236402560', '2051105475239023232',
       '2051105479530829184', '2051105509596145408', '2051105475236399232',
       '2051105509596144256', '2051105513890642176', '2051105479531687808',
       '2051105513895456896', '2051105479535690112', '2051105406514989312',
       '2051105513895457664', '2051105479530816256', '2051105410811369728',
       '2051105445171164928', '2051105513895452928', '2051105479531687936',
       '2051105410816218368', '2051105509593807616', '2051105440876663936',
       '2051105685694143104', '2051105406516922240', '2051105513895463040',
       '2051

In [9]:
%time

print(wocs_63060.export_features(dump=False))

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.72 µs
{
    "ID": "2051105479535706752",
    "ra": 290.294601905,
    "dec": 37.7587177584,
    "Gaia_mag": 15.57,
    "BP_mag": 15.82,
    "BP_RP": 0.63,
    "BP_G": 0.24,
    "G_RP": 0.38,
    "ndet": 4193,
    "mintime": 443.9396820654948,
    "maxtime": 537.631896822495,
    "timelength": 93.69221475700022,
    "amplitude": 0.013280683970898544,
    "ndetobslength_ratio": 44.752917954548835,
    "linear_fit_slope": 15.58231711883195,
    "magnitude_ratio": 0.5251439251941965,
    "median": 15.58328681960397,
    "wmean": 15.583250769783442,
    "mad": 0.0027228738707734124,
    "stdev": 0.004038021950356971,
    "skew": -0.053266806987786824,
    "kurtosis": -0.2584735886194469,
    "beyond1std": 0.30932506558549966,
    "mag_iqr": 0.0054423651105395265,
    "eta_normal": 0.42793365298731,
    "eta_robust": 0.02379123588671188,
    "eta_uneven_normal": 8544101.59558457,
    "eta_uneven_robust": 2560938536.6056404,
    "f

In [10]:
wocs_63060.RLC.BLS

{'bestperiod': 0.7232568175965727,
 'bestlspval': 0.0022862207155785073,
 'nbestpeaks': 3,
 'nbestlspvals': [0.0022862207155785073,
  0.0017773305459861596,
  0.0016532705431525291],
 'nbestperiods': [0.7232568175965727, 1.446434238504079, 2.890431587000283],
 'lspvals': array([0.00041692, 0.00040709, 0.00036966, ..., 0.00012153, 0.00012424,
        0.0001589 ]),
 'frequencies': array([ 0.05      ,  0.05002668,  0.05005337, ..., 23.80944475,
        23.80947143, 23.80949812]),
 'periods': array([20.        , 19.98933245, 19.97867627, ...,  0.04200014,
         0.04200009,  0.04200005]),
 'blsresult': [{'power': array([0.00041692, 0.00040709, 0.00036966, ..., 0.00010594, 0.00010037,
          0.00010365]),
   'bestperiod': 0.7232568175965727,
   'bestpower': 0.0022862207155785073,
   'transdepth': 0.004650078717658983,
   'transduration': 0.4090150250417362,
   'transingressbin': 52,
   'transegressbin': 133},
  {'power': array([0.00011619, 0.00010679, 0.00011058, ..., 0.00012153, 0.000

In [11]:
wocs_63060.export_features(dump=True, per_period=True)

In [12]:
wocs_63060.RLC.Stwf

{'bestperiod': 0.723577992960771,
 'bestlspval': 0.5791194854549672,
 'nbestpeaks': 3,
 'nbestlspvals': [0.5791194854549672, 0.5902022453101254, 0.6101459834672595],
 'nbestperiods': [0.723577992960771,
  0.042058114647552214,
  0.05962019234347414],
 'lspvals': array([0.99817542, 0.99878871, 1.00148138, ..., 1.00136805, 0.9988669 ,
        0.99982351]),
 'periods': array([20.        , 19.18110158, 18.42662489, ...,  0.04200908,
         0.04200532,  0.04200155]),
 'method': 'pdm',
 'kwargs': {'startp': 0.042,
  'endp': 20.0,
  'stepsize': 0.001,
  'normalize': False,
  'phasebinsize': 0.05,
  'mindetperbin': 9,
  'autofreq': True,
  'periodepsilon': 0.1,
  'nbestpeaks': 3,
  'sigclip': 5.0},
 'nbestpeakratios': [array([432.64828698]),
  array([302.63924871]),
  array([310.72960856])],
 'siderealflags': [True, True, True]}

In [ ]:
MakeStardict(export_all=True, cluster='ngc6791')

In [ ]:
stardict

In [ ]:
a={'a':1, 'b':2}
dd={**a, 'v':100}

type(np.nan)